## Usando arquivo .env para controlar variaveis de ambiente
Para evitar exposição da chave `OPENAI_API_KEY` optei por utilizar arquivo `.env` com a informação da chave.

Para seguir o mesmo método basta criar um arquivo `.env` no mesmo diretório do arquivo `parentRAG.ipynb`.
A importação da chave será feita através da célula abaixo que faz a instalação de um biblioteca para carregar
os valores do arquivo `.env`.

In [4]:
%pip install python-dotenv
import dotenv
%load_ext dotenv
%dotenv


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [22]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma
from langchain_community.document_loaders import PyPDFLoader

from langchain_core.prompts import ChatPromptTemplate

from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_core.output_parsers import StrOutputParser

from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryByteStore


import os
# os.environ["OPENAI_API_KEY"] = "" EDITE esse trecho e coloque sua key caso queira

In [7]:
#
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
llm = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    max_tokens=500,
)


In [9]:
# Carregar o PDF
pdf_link = "DOC-SF238339076816-20230503.pdf"
loader = PyPDFLoader(pdf_link, extract_images=False)
pages = loader.load_and_split()

In [10]:
len(pages)

31

In [11]:
# Sppliter

chield_splitter = RecursiveCharacterTextSplitter(chunk_size=200)
parent_splitter = RecursiveCharacterTextSplitter(
    chunk_size=4000,
    chunk_overlap=200,
    length_function=len,
    add_start_index=True
)

In [13]:
# Storages
store = InMemoryByteStore()
vectorstore = Chroma(embedding_function=embeddings, persist_directory="chieldVectorDB")

In [15]:
parent_document_retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=chield_splitter,
    parent_splitter=parent_splitter,
)

parent_document_retriever.add_documents(pages, ids=None)

In [16]:
parent_document_retriever.vectorstore.get()

{'ids': ['195e67dd-0ac8-4396-b744-689ad7828164',
  'b4ca1d50-1831-4bd5-b6bb-20d5be0a0166',
  'eea18bea-6de2-41f9-8260-1a2498660641',
  'da6aaa1e-232b-4fd2-b3ee-c4b773d770ac',
  '6e8cc912-0b8a-4885-9545-420aa4292942',
  '3b7f75aa-b5c7-4bb6-9fae-1c17525a4898',
  'a33c4e77-893d-4f40-95ca-acaebd79992d',
  '9008f6fe-cad7-45ec-bb29-4436f1709d93',
  '0962ce71-e2be-4196-9c80-0ef725805694',
  '80eba4bf-7b7d-42c5-b5e1-2d83429114cd',
  '753e39fe-cff4-4f75-9b92-23a6c907954e',
  '431963c7-8f64-4472-83f9-8c14f676d3e3',
  '0d8af08e-7fb5-496c-a0c8-1d549884467e',
  'f6a4c961-fa07-4e04-9fb4-0751388c8fc9',
  'd6dbf876-8abf-4fd7-b278-8f9ead264980',
  '9cbee391-6cde-4712-b8d4-794454eed27d',
  '4fb74596-4d82-4bc6-8cb5-a0f547fae7de',
  '7e9a90d0-f2b1-4e29-85d0-983c3258462f',
  '633d713b-2193-44c5-adf6-8eeb0e7e6d3a',
  'a09e7509-dcbf-4be3-b13b-89fffe47a2be',
  'f028e838-d653-4ec5-a9c9-1af50ac2cf9e',
  'af6adf9c-f77f-4afb-9485-514eeeea8155',
  'e0b8bdfe-3d8e-44fa-9499-dc8bc9e97ae3',
  'bf7ee27c-a03f-49e5-a307-

In [18]:
TEMPLATE = """
    Você é um especialista em legislação e tecnologia. Responda a pergunta abaixo utilizando o contexto informado.
    Query:
    {question}
    Context:
    {context}
"""

In [19]:
rag_prompt = ChatPromptTemplate.from_template(TEMPLATE)

In [23]:
setup_retrival = RunnableParallel({"question": RunnablePassthrough(), "context": parent_document_retriever})
output_parser = StrOutputParser()

In [24]:
parent_chain_retrieval = setup_retrival | rag_prompt | llm | output_parser

In [25]:
parent_chain_retrieval.invoke("Quais os principais riscos do marco legal de ia")

'Os principais riscos do marco legal de IA incluem a possibilidade de discriminação, a falta de transparência nos algoritmos utilizados, a responsabilização civil em casos de danos causados por sistemas de IA e a necessidade de fiscalização eficaz para garantir o cumprimento das normas estabelecidas. Além disso, é importante considerar a categorização dos riscos da inteligência artificial, a avaliação preliminar das aplicações, a definição de aplicações de alto risco e a necessidade de medidas adicionais para garantir a transparência e mitigar vieses nos sistemas de IA.'